In [ ]:
import io
import os
import tempfile
from pathlib import Path

from rich import print as rprint

from entitysdk.client import Client
from entitysdk.common import ProjectContext
from entitysdk.models import (
    BrainLocation,
    BrainRegion,
    CellMorphology,
    CellMorphologyProtocol,
    Contribution,
    MTypeClass,
    MTypeClassification,
    Organization,
    Role,
    Species,
    Strain,
    Subject,
)
from entitysdk.types import (
    CellMorphologyGenerationType,
    CellMorphologyProtocolDesign,
    SlicingDirectionType,
)

### Initialize the entitycore client

In [ ]:
entitycore_api_url = "http://127.0.0.1:8000"
project_context = ProjectContext(
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730",
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6",
)
token = os.getenv("ACCESS_TOKEN", "XXX")
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)


# uncomment for staging
# from obi_auth import get_token
# token = get_token(environment="staging")
# Replace this with your vlab project url in staging
# project_context = ProjectContext.from_vlab_url("https://staging.openbraininstitute.org/app/virtual-lab/lab/594fd60d-7a38-436f-939d-500feaa13bba/project/ff89ca07-6613-4922-9ab0-2637221db8b5/home")
# client = Client(environment="staging", project_context=project_context)

## Search for entities

In [ ]:
species = client.search_entity(entity_type=Species, query={"name": "Mus musculus"}, limit=10).one()

In [ ]:
rprint(species)

In [ ]:
strain = client.search_entity(entity_type=Strain, query={"name": "Cux2-CreERT2"}).one()

In [ ]:
rprint(strain)

In [ ]:
brain_region = client.search_entity(entity_type=BrainRegion, query={"annotation_value": 68}).one()

In [ ]:
rprint(brain_region)

### Create and register a new subject

Subjects should be reused if possible, but they can be created when needed.

In [ ]:
subject = Subject(
    name="my-subject-for-morphology",
    description="my-subject-description",
    sex="male",
    species=species,
    strain=strain,
)

subject = client.register_entity(subject)
rprint(subject)

### Create and register a new morphology protocol

Morphology protocols should be reused if possible, but they can be created when needed.

In [ ]:
morphology_protocol = CellMorphologyProtocol(
  generation_type=CellMorphologyGenerationType.digital_reconstruction,
  protocol_document="https://example.com/",
  protocol_design=CellMorphologyProtocolDesign.cell_patch,
  slicing_thickness=20.0,
  slicing_direction=SlicingDirectionType.horizontal,
)
rprint(morphology_protocol.__class__.__name__)
morphology_protocol = client.register_entity(morphology_protocol)
rprint(morphology_protocol)

### Optional, list morphology protocols

In [ ]:
protocols = client.search_entity(entity_type=CellMorphologyProtocol).all()
rprint(protocols)

### Create a morphology object

In [ ]:
brain_location = BrainLocation(
    x=4101.52490234375,
    y=1173.8499755859375,
    z=4744.60009765625,
)
morphology = CellMorphology(
    cell_morphology_protocol=morphology_protocol,
    name="my-morph",
    description="A morphology",
    subject=subject,
    brain_region=brain_region,
    location=brain_location,
    legacy_id=None,
    authorized_public=False,
)

In [ ]:
rprint(morphology)

## Register morphology

In [ ]:
registered = client.register_entity(entity=morphology)

In [ ]:
rprint(registered)

## Add mtype to morphology

In [ ]:
mtype = client.search_entity(entity_type=MTypeClass, query={"pref_label": "L5_PC"}).one()

mtype_classification = MTypeClassification(
    mtype_class_id=mtype.id, entity_id=registered.id, authorized_public=True
)

mtype_classification = client.register_entity(mtype_classification)

## Add contribution to morphology

In [ ]:
agent = client.search_entity(entity_type=Organization, limit=1).one()
role = client.search_entity(entity_type=Role, limit=1).one()
contribution = Contribution(
    agent=agent,
    role=role,
    entity=registered,
)
contribution = client.register_entity(contribution)

In [ ]:
rprint(contribution)

## Upload assets

In [ ]:
with tempfile.TemporaryDirectory() as tdir:
    file1 = Path(tdir, "morph.h5")
    file1.write_text("h5")

    file2 = Path(tdir, "morph.swc")
    file2.write_text("swc")

    # use a filepath to register first asset
    asset1 = client.upload_file(
        entity_id=registered.id,
        entity_type=CellMorphology,
        file_path=file1,
        file_content_type="application/x-hdf5",
        asset_label="morphology",
    )
    rprint(asset1)

    # use an in-memory buffer to upload second asset
    buffer = io.BytesIO(b"morph bytes buffer")

    asset2 = client.upload_content(
        entity_id=registered.id,
        entity_type=CellMorphology,
        file_content=buffer,
        file_name="buffer.swc",
        file_content_type="application/swc",
        asset_label="morphology",
    )
    rprint(asset2)

## Retrieve it

In [ ]:
# with assets and mtypes
fetched = client.get_entity(entity_id=registered.id, entity_type=CellMorphology)

## Download asset

In [ ]:
downloaded = client.download_assets(
    fetched,
    selection={"content_type": "application/swc"},
    output_path="./my-file.swc",
).one()

content = client.download_content(
    entity_id=fetched.id, entity_type=type(fetched), asset_id=downloaded.asset.id
)

print(content)
print(Path("my-file.h5").read_text())

## Delete asset

In [ ]:
for asset in fetched.assets:
    if asset.content_type == "application/swc":
        print("Deleting asset ", asset.id)
        deleted_asset = client.delete_asset(
            entity_id=fetched.id,
            entity_type=type(registered),
            asset_id=asset.id,
        )
        break

rprint(deleted_asset)

fetched = client.get_entity(entity_id=registered.id, entity_type=CellMorphology)
rprint(fetched.assets)

## Search it

In [ ]:
hits = client.search_entity(
    entity_type=CellMorphology,
    query={"name__ilike": "my-morph", "page": 1, "page_size": 2},
    limit=None,
).all()

print("Number of results: ", len(hits))

In [ ]:
rprint(hits[0])

## Register existing data from an external cloud storage

In [ ]:
## register a file
# asset = client.register_asset(
#     entity_id=registered.id,
#     entity_type=CellMorphology,
#     name="my-morphology.h5",
#     storage_path="path/to/morph.h5",
#     storage_type="aws_s3_open",
#     is_directory=False,
#     content_type="application/x-hdf5",
#     asset_label="morphology",
# )

In [ ]:
## register a directory
# asset = client.register_asset(
#     entity_id=registered.id,
#     entity_type=Circuit,
#     name="sonata_circuit",
#     storage_path="path/to/circuit/directory",
#     storage_type="aws_s3_open",
#     is_directory=True,
#     content_type="application/vnd.directory",
#     asset_label="sonata_circuit",
# )